In [5]:
!git clone https://github.com/PhiCtl/isospec-internship/

fatal: destination path 'isospec-internship' already exists and is not an empty directory.


In [16]:
import warnings
warnings.filterwarnings("ignore")
from IPython.display import HTML
from nbdev.showdoc import show_doc

In [6]:
import numpy as np
import pandas as pd
import os

In [21]:
# Ref https://github.com/BojarLab/SweetNet/blob/main/SweetNet_code.ipynb

In [ ]:
!pip install glycowork nbdev

In [19]:
!pip install torch-geometric torch-sparse torch-scatter torch-cluster torch-spline-conv -f https://pytorch-geometric.com/whl/torch-1.7.0.html

Looking in links: https://pytorch-geometric.com/whl/torch-1.7.0.html
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 210.0/210.0 kB 4.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 108.0/108.0 kB 9.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.5/54.5 kB 4.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done


In [ ]:
from torch_geometric.nn import TopKPooling, GraphConv
from torch_geometric.nn import global_mean_pool as gap, global_max_pool as gmp
import torch.nn.functional as F

In [ ]:
import torch

In [13]:
from glycowork.ml.models import *
from glycowork.ml.inference import *
from glycowork.ml.processing import *
from glycowork.ml.model_training import *
from glycowork.ml.train_test_split import *
from glycowork.glycan_data.loader import df_species, df_glycan
#%load_ext autoreload

In [7]:
DATA_PATH = "/content/isospec-internship/data/glycan_embedding"

# Datasets

In [8]:
# Extracted glycans used for inference and enrichment
df_glycan_list = pd.read_csv(os.path.join(DATA_PATH, 'glycan_list.csv'))
df_glycan_list.head()

,glycan,Composition,tissue_species,tissue_sample
0,Fuc(a1-?)GlcNAc(b1-2)Man(a1-6)[GlcNAc(b1-2)Man...,"{'dHex': 2, 'HexNAc': 4, 'Hex': 3}",['Homo_sapiens'],['blood']
1,Neu5Ac(a2-?)Gal(b1-4)GlcNAc(b1-2)Man(a1-6)[Glc...,"{'Neu5Ac': 1, 'Hex': 4, 'HexNAc': 4, 'dHex': 1}",['Homo_sapiens'],['blood']
2,Neu5Ac(a2-6)Gal(b1-4)GlcNAc(b1-2)Man(a1-6)[Gal...,"{'Neu5Ac': 1, 'Hex': 5, 'HexNAc': 4}",['Homo_sapiens'],['blood']
3,Neu5Ac(a2-6)Gal(b1-4)GlcNAc(b1-2)Man(a1-6)[Glc...,"{'Neu5Ac': 1, 'Hex': 4, 'HexNAc': 4}",['Homo_sapiens'],['blood']
4,Fuc(a1-2)[GalNAc(a1-3)]Gal(b1-4)GlcNAc(b1-2)Ma...,"{'dHex': 1, 'HexNAc': 5, 'Hex': 5}",['Homo_sapiens'],['blood']


In [14]:
# Glycan sequences used for embedding space learning
df_glycan_list = pd.read_pickle(os.path.join(DATA_PATH, 'df_glycan.pkl'))

In [9]:
df_glycan_binding = pd.read_pickle(os.path.join(DATA_PATH, 'glycan_binding.pkl'))
df_glycan_binding.head(10)

,3-Anhydro-Gal(a1-3)Gal(b1-4)3-Anhydro-Gal(a1-3)Gal4S,3-Anhydro-Gal(a1-3)Gal4S(b1-4)3-Anhydro-Gal(a1-3)Gal4S,3-Anhydro-Gal(a1-3)Gal4S(b1-4)3-Anhydro-Gal(a1-3)Gal4S(b1-4)3-Anhydro-Gal(a1-3)Gal4S,3-Anhydro-Gal(a1-3)Gal4S(b1-4)3-Anhydro-Gal(a1-3)Gal4S(b1-4)3-Anhydro-Gal(a1-3)Gal4S(b1-4)3-Anhydro-Gal(a1-3)Gal4S,3-Anhydro-Gal(a1-3)Gal4S(b1-4)3-Anhydro-Gal2S(a1-3)Gal4S(b1-4)3-Anhydro-Gal(a1-3)Gal4S,3dGal(b1-3)[Fuc(a1-4)]Glc,3dGal(b1-4)Glc,4d8dNeu5Ac(a2-3)Gal(b1-4)Glc,4dNeu5Ac(a2-3)Gal(b1-4)Glc,7dNeu5Ac(a2-3)Gal(b1-4)Glc,...,wwwSflexneri5c,wwwSflexneriO2c,wwwSflexneriO5c,wwwSisomicin,wwwSmix,wwwTobramycin,wwwTyrS,wwwpHGGs,target,protein
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,AADSIPSISPTGIITPTPTQSGMVSNCNKFYDVHSNDGCSAIASSQ...,TAL6-4LysM
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,AAFFSLVVLLALLPFGIHASALPSTELTPRVNPNLPGPNDVFVGFR...,rCnSL-proA
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,AANEADYQAKLTAYQTELARVQKANADAKAAYEAAVAANNAANAAL...,AntigenI/IIA3VP1
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,AASKLGVPQPAQRDQVNCQLYAVQPNDNCIDISSKNNITYAQLLSW...,TAL6-6LysM
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,ACNNEWEDEQYEQYISFKSPIPAGGEGVTDIYVRYKEDGKVTYRLP...,SP15308A-bot-339-19-339
5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,ACPSQCSCSGTEVNCAGKSLASVPAGIPTTTRVLYLNSNQITKLEP...,VLRB.aGPA.23-GCN4-biotin
6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,ADEPIDLEKLEEKRDKENVGNLPKFDNEVKDGSENPMAKYPDFDDE...,Protein L(A-C2)
7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,ADGIQDKICIGYLSNNSTDTVDTLTENGVPVTSSIDLVETNHTGTY...,A/H16-2
8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,ADGIQDKICIGYLSNNSTDTVDTLTENGVPVTSSIDLVETNHTGTY...,A/H16
9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,ADGIQDRICVGYLSTNSSERVDTLLENGVPVTSSIDLIETNHTGTY...,A/Gull/Maryland/704/1977(H13N6)


In [11]:
# Used to control representation in the embedding space by assessing closeness
df_N_glycans = pd.read_pickle(os.path.join(DATA_PATH, 'N_glycans_df.pkl'))
df_N_glycans.head()

,glycan,Species,Genus,Family,Order,Class,Phylum,Kingdom,Domain,ref,...,disease_sample,disease_direction,disease_ref,disease_species,tissue_sample,tissue_id,tissue_ref,tissue_species,Composition,Structure_Type
1,Neu5Ac(a2-3)Gal(b1-4)GlcNAc(b1-2)Man(a1-6)[Gal...,"[Cricetulus_griseus, Homo_sapiens, Mus_musculu...","[Cricetulus, Homo, Mus, Ovis, Rattus]","[Cricetidae, Hominidae, Muridae, Bovidae, Muri...","[Rodentia, Primates, Rodentia, Artiodactyla, R...","[Mammalia, Mammalia, Mammalia, Mammalia, Mamma...","[Chordata, Chordata, Chordata, Chordata, Chord...","[Animalia, Animalia, Animalia, Animalia, Anima...","[Eukarya, Eukarya, Eukarya, Eukarya, Eukarya]","[, , https://unicarb-dr.glycosmos.org/referenc...",...,[],[],[],[],"[2A3_cell_line, AML_193_cell_line, Cal-27_cell...","[cellosaurus:CVCL_0D71, cellosaurus:CVCL_1071,...","[https://pubmed.ncbi.nlm.nih.gov/36289103/, ht...","[Homo_sapiens, Homo_sapiens, Homo_sapiens, Hom...","{'Neu5Ac': 1, 'Hex': 5, 'HexNAc': 4}",Complex_Gal
2,Gal(a1-3)Gal(b1-4)GlcNAc(b1-2)Man(a1-6)[Gal(b1...,"[Bos_taurus, Ginglymostoma_cirratum, Mus_muscu...","[Bos, Ginglymostoma, Mus, Sus]","[Bovidae, Ginglymostomatidae, Muridae, Suidae]","[Artiodactyla, Orectolobiformes, Rodentia, Art...","[Mammalia, Chondrichthyes, Mammalia, Mammalia]","[Chordata, Chordata, Chordata, Chordata]","[Animalia, Animalia, Animalia, Animalia]","[Eukarya, Eukarya, Eukarya, Eukarya]","[, https://pubmed.ncbi.nlm.nih.gov/19156518/, ...",...,[],[],[],[],[],[],[],[],"{'Hex': 6, 'HexNAc': 4}",Complex_Gal
3,Neu5Ac(a2-?)Gal(b1-4)GlcNAc(b1-2)Man(a1-6)[Gal...,"[Notamacropus_eugenii, Homo_sapiens, Homo_sapi...","[Notamacropus, Homo, Homo]","[Macropodidae, Hominidae, Hominidae]","[Diprotodontia, Primates, Primates]","[Mammalia, Mammalia, Mammalia]","[Chordata, Chordata, Chordata]","[Animalia, Animalia, Animalia]","[Eukarya, Eukarya, Eukarya]","[https://pubmed.ncbi.nlm.nih.gov/23053637/, ht...",...,"[, ]","[, ]","[, ]","[, ]","[urine, urine]","[UBERON:0001088, UBERON:0001088]","[https://pubmed.ncbi.nlm.nih.gov/33650863/, ht...","[Homo_sapiens, Homo_sapiens]","{'Neu5Ac': 1, 'Hex': 5, 'HexNAc': 4}",Complex_Gal
4,Fuc(a1-2)Gal(b1-4)GlcNAc(b1-2)Man(a1-6)[Gal(b1...,[Homo_sapiens],[Homo],[Hominidae],[Primates],[Mammalia],[Chordata],[Animalia],[Eukarya],[],...,[],[],[],[],[plasma],[ENVO:01000798],[https://pubmed.ncbi.nlm.nih.gov/1577715/],[Homo_sapiens],"{'dHex': 1, 'Hex': 5, 'HexNAc': 4}",Complex_Gal
5,Fuc(a1-2)[Gal(a1-3)]Gal(b1-4)GlcNAc(b1-2)Man(a...,[Homo_sapiens],[Homo],[Hominidae],[Primates],[Mammalia],[Chordata],[Animalia],[Eukarya],[],...,[],[],[],[],[plasma],[ENVO:01000798],[https://pubmed.ncbi.nlm.nih.gov/1577715/],[Homo_sapiens],"{'dHex': 1, 'Hex': 6, 'HexNAc': 4}",Complex_Gal
